In [1]:
from pymongo import MongoClient
import pymongo
import pandas as pd
import numpy as np
import src.tools.mongo as mongo
import src.tools.geo as geo
import src.tools.google as google
import time
import dotenv
import os
dotenv.load_dotenv()
key = os.getenv("GOOGLE_KEY")

Connect to Mongo

In [2]:
client=mongo.localConection()
db = client.companies

Create new collection

In [3]:
facilities=db["facilities"]

Eliminar puntos lejanos de la ciudad o con la latitud incorrecta    

In [22]:
cities=pd.DataFrame(db.offices.find({},{"city":1,"_id":0}))
cities=cities[~cities.duplicated()]

Get all the cities

In [ ]:
for city in cities.city:
    time.sleep(1.5)
    db.offices.delete_many(mongo.farPoints(city, 10))

In [27]:
cities=pd.DataFrame(db.offices.find({},{"city":1,"latitude":1,"longitude":1,"_id":0}))

In [40]:
cities=cities.groupby("city").mean()

In [31]:
def geopoint(long, lat): 
    return {'type': 'Point', 'coordinates': [long,lat]}

cities["geopoint"]=np.vectorize(geopoint)(cities["longitude"],cities["latitude"])

client=mongo.localConection()
db = client.companies
coll=db["prueba"] 
coll.insert_many(cities.to_dict('record'))
coll.create_index([('geopoint', pymongo.GEOSPHERE)])

'geopoint_2dsphere'

In [7]:
#Buscar y guradar info de la ciudad
#name ranking coord

In [82]:
school=[]
for i in range(3):
    print(cities.iloc[i].latitude,cities.iloc[i].longitude)
    aux=google.getNearbyplaces(cities.iloc[i].latitude,cities.iloc[i].longitude,radius=15000,googleType="school")
    print("=====")
    print(i)
    print(len(aux))
    school+=aux

33.766154047368424 -84.37878671052631
=====
0
60
30.2705520088889 -97.76187868888891
=====
1
60
52.51623264516128 13.400849138709676
=====
2
60


In [83]:
df_school=pd.DataFrame(school)
df_school["geopoint"]=np.vectorize(geopoint)(df_school["longitude"],df_school["latitude"])

client=mongo.localConection()
db = client.companies
coll=db["prueba2"] 
coll.insert_many(df_school.to_dict('record'))
coll.create_index([('geopoint', pymongo.GEOSPHERE)])

'geopoint_2dsphere'

In [80]:
df_school

,name,latitude,longitude,rating,geopoint
0,David Lloyd Cambridge,52.193778,0.167308,3.5,"{'type': 'Point', 'coordinates': [0.1673081, 5..."
1,The Leys School,52.196595,0.120370,4.2,"{'type': 'Point', 'coordinates': [0.1203697, 5..."
2,Sawston Village College,52.126230,0.164750,4.3,"{'type': 'Point', 'coordinates': [0.16475, 52...."
3,Chesterton Sports Centre,52.217649,0.121782,4.0,"{'type': 'Point', 'coordinates': [0.121782, 52..."
4,Saint Phillip's Church of England Primary School,52.200505,0.154186,3.0,"{'type': 'Point', 'coordinates': [0.154186, 52..."
5,"Churchill College, University of Cambridge",52.213130,0.099870,4.6,"{'type': 'Point', 'coordinates': [0.09987, 52...."
6,Coleridge School,52.192750,0.152400,NaN,"{'type': 'Point', 'coordinates': [0.1524003, 5..."
7,Scallywags Day Nursery,52.238777,0.084028,5.0,"{'type': 'Point', 'coordinates': [0.0840283, 5..."
8,Scallywags,52.267834,0.081309,4.8,"{'type': 'Point', 'coordinates': [0.0813089999..."
9,Hills Road Sports & Tennis Centre,52.187323,0.134940,4.4,"{'type': 'Point', 'coordinates': [0.1349405, 5..."


In [11]:
b["geopoint"]=np.vectorize(mongo.geopoint)(b["longitude"],b["latitude"])

In [12]:
b.head()

,name,latitude,longitude,rating,geopoint
0,Manhattan School of Music,40.812632,-73.961679,4.4,"{'type': 'Point', 'coordinates': [-73.9616791,..."
1,Stuyvesant High School,40.717758,-74.013864,4.2,"{'type': 'Point', 'coordinates': [-74.0138644,..."
2,Forest Hills High School,40.729712,-73.844926,4.0,"{'type': 'Point', 'coordinates': [-73.84492589..."
3,Long Island City High School,40.765470,-73.933053,3.9,"{'type': 'Point', 'coordinates': [-73.93305289..."
4,Beacon School,40.761052,-73.996472,3.9,"{'type': 'Point', 'coordinates': [-73.9964725,..."


In [13]:
client=mongo.localConection()
db = client.companies
coll=db["facilities"] 
coll.insert_many(b.to_dict('record'))
coll.create_index([('geopoint', pymongo.GEOSPHERE)])

'geopoint_2dsphere'